# Data Overview Example

- What it could look like
- The final DataFrame could be stored in a csv file which can then be used in Excel other programs

In [1]:
%cd ..

/home/konrad/programming/python/correlaid/datenguide-python


In [2]:
from datenguidepy.query_helper import get_all_regions, get_statistics
from datenguidepy import Query

import pandas as pd
import numpy as np


pd.set_option('display.max_colwidth', 150)

In [3]:
BL = get_all_regions().query("level == 'nuts1'")

In [4]:
bl_codes = BL.index.tolist()
bl_names = BL.name.tolist()

### Creating a DataFrame with information - for 1 statistic
- My idea was to cycle through different options and pull out only descriptive information from the run queries, to understand:
    - the shape of resulting DataFrame
    - the years containing information
    - additional args available
    - validity and periodicity
- As the first step I started with pulling information for one statistic for all Bundesländer
- Then I ran into an issue with the dimensions

<span style="color: blue"> I think the idea is very good but would try to focus a little more on coverage. Which in turn would aim at methods specifically designed to give results for all regions and statistics. In order for this not to get too complicated I would reduce the descriptive information to begin with. </span>

In [5]:
#create a list of df
df_lst = []

for bl in bl_codes:
    bl_q = Query.region(str(bl))
    bl_q.add_field('PLZ005')
    
    # create dataframe and store in a list
    df_bl = bl_q.results()
    df_lst.append(df_bl)

/home/konrad/programming/python/correlaid/datenguide-python/datenguidepy/output_transformer.py:130: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  return pd.io.json.json_normalize(statistic_sub_json, sep="_", max_level=1)


In [6]:
# Going through the DF
rows_lst = []
clmns_lst = []
yr_min = []
yr_max = []
yr_diff = []
s_valid = []
s_period = []

for df in df_lst:
    # shape
    rws = df.shape[0]
    clmns = df.shape[1]
    rows_lst.append(rws)
    clmns_lst.append(clmns)
    
    #years
    ymin = df.year.min()
    ymax = df.year.max()
    diff = ymax - ymin
    yr_min.append(ymin)
    yr_max.append(ymax)
    yr_diff.append(diff)
    
    #source
    valid = df.iat[0,5]
    period = df.iat[0,6]
    s_valid.append(valid)
    s_period.append(period)

<span style = "color: blue">
<h3> Suggestion for reduced statistics</h3>
I recommend reduce the number of stored values for now. My suggestion is:
    
<ul>
<li> Remove Columns as this is related to query building itself (I can add or remove columns)</li>
<li> Without a Columns column I would rename Rows into Entries </li>
<li> Eventually I would remove BL as a technichal overview does not require it and application and use cases can be build to reconstruct it from the Code</li>
<li> I would "Source valid from" and "Source periodicity" as they are hard to put into perspective for a new user. </li>
<li> I would add "Number of distinct sources" (as the unique number of source_names) as that might have direct user relevance (2 entries for one year)</li>
<li> I would drop Total years for now for simplicity. The reason is that the number might easily be scewed when there are multiple sources for which one reports the statistic on a yearly basis and the other reports it for instance every 4 years. In simpler situations where there is not multiple sources per statistic, this number should be the same as Entries.</li>
<li> Unlike the excel file, I would not include args/enums in the analysis yet for simplicity. Although this is potentially very interesting, these vary a lot across statistics and a I think I will be quite some further work to design and obtain a good overview for these. Also an analysis of these will possibly be easier in the near future when some of the enhancement issues have been implemented.</li>
</ul>

    
In terms of analysis structure I recommend to create a function that calculates all the descriptive information for a single query result as a tuple or dictionary.
When looping over several results this allows for easier collection of the results as a list of tuples/dicts which can be conveniently converted into a DataFrame
    
</span>



In [7]:
def summarize_result(region,statistic,query_result):
    summary = {}
    summary['region'] = region
    summary['statistic'] = statistic
    if not query_result.empty:
        summary['region_name'] = query_result.name.iloc[0] #temporarily for convenience
        summary['entries'] = query_result.shape[0]
        summary['start_year'] = query_result.year.min()
        summary['end_year'] = query_result.year.max()
        summary['distinct_sources'] = query_result[f'{statistic}_source_name'].nunique()
    else:
        summary['entries'] = 0
        # missing dict entries in this case will automatically
        # create NaN entries when converted to a pandas DataFrame
    return summary

In [8]:
results = pd.DataFrame ({'Code': bl_codes, 'BL': bl_names, 'Statistic': 'PLZ005', 
                          'Rows' : rows_lst, 
                          'Columns': clmns_lst,
                          'Start year': yr_min,
                          'End year' : yr_max,
                          'Total years' : yr_diff,
                          'Source valid from' : s_valid,
                          'Source periodicity' : s_period
                         })
results

,Code,BL,Statistic,Rows,Columns,Start year,End year,Total years,Source valid from,Source periodicity
0,10,Saarland,PLZ005,25,9,2007,2019,12,1994-01-01T00:00:00,VIERJAEHRLICH
1,11,Berlin,PLZ005,25,9,2007,2019,12,1994-01-01T00:00:00,VIERJAEHRLICH
2,12,Brandenburg,PLZ005,25,9,2007,2019,12,1994-01-01T00:00:00,VIERJAEHRLICH
3,13,Mecklenburg-Vorpommern,PLZ005,25,9,2007,2019,12,1994-01-01T00:00:00,VIERJAEHRLICH
4,14,Sachsen,PLZ005,25,9,2007,2019,12,1994-01-01T00:00:00,VIERJAEHRLICH
5,15,Sachsen-Anhalt,PLZ005,25,9,2007,2019,12,1994-01-01T00:00:00,VIERJAEHRLICH
6,16,Thüringen,PLZ005,25,9,2007,2019,12,1994-01-01T00:00:00,VIERJAEHRLICH
7,01,Schleswig-Holstein,PLZ005,25,9,2007,2019,12,1994-01-01T00:00:00,VIERJAEHRLICH
8,02,Hamburg,PLZ005,25,9,2007,2019,12,1994-01-01T00:00:00,VIERJAEHRLICH
9,03,Niedersachsen,PLZ005,25,9,2007,2019,12,1994-01-01T00:00:00,VIERJAEHRLICH


### Creating a DataFrame with information - for more statistics

<span style="color: blue"> 
<p> 
    I for one this seems to potentially have uncovered a bug in the underlying joining logic. Thats good and I'll try to folow up on it.
    It seems to be related to TIE003 being empty at the same time when our package tries to join it with other data.
</p>
    
<p>
    Even when it works I don't think it is so important to join the different statsitics using our package functionality, as this will
    skew row counts. One reason is the outer join character of the query functinality. The other reason is if a statistic has multiple
    values during the same year, joining migh multiply the number of rows in total. When directly querying a statistic I don't think
    the API should provide more than one value a year (if it works correctly), but it nonethelesse sometimes does. As in the case
    of PLZ005 for instance. As such I reccomond creating a single query for every region,stat combination. See example blow.
</p>
</span>



In [9]:
# Random selection of stats for test run

stats_lst = ['PLZ005','BEVSTD' ]# Works but not advisable
# stats_lst = ['PLZ005', 'TIE003', 'BEVSTD' ]

In [10]:
# failed attempt to create a second list of DataFrames
df_lst2 = []

for bl in bl_codes:
    bl_qu = Query.region(str(bl))
    
    for stat in stats_lst:
        bl_qu.add_field(str(stat))
        
    
    # create dataframe and store in a list
    df_bl = bl_qu.results()
    df_lst2.append(df_bl)

/home/konrad/programming/python/correlaid/datenguide-python/datenguidepy/output_transformer.py:130: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  return pd.io.json.json_normalize(statistic_sub_json, sep="_", max_level=1)


### Create a query for each combination

In [11]:

stats_lst = ['PLZ005', 'TIE003', 'BEVSTD' ]

query_summaries = []
for region in bl_codes:
    for stat in stats_lst:
        q = Query.region(region)
        q.add_field(stat)
        query_result = q.results()
        query_summaries.append(summarize_result(region,stat,query_result))
        
summary_frame = pd.DataFrame(query_summaries)
summary_frame.head()

,region,statistic,region_name,entries,start_year,end_year,distinct_sources
0,10,PLZ005,Saarland,25,2007.0,2019.0,1.0
1,10,TIE003,NaN,0,NaN,NaN,NaN
2,10,BEVSTD,Saarland,31,1995.0,2018.0,2.0
3,11,PLZ005,Berlin,25,2007.0,2019.0,1.0
4,11,TIE003,NaN,0,NaN,NaN,NaN


<span style="color: blue">

### Non datenguide related extra information

A slightly more pythonian (although completely equivalent) way of dealing with nested for-loops is to use the `product` function from the `itertools` module. This is good practice for readability particularlrly when dealing with more then 2 nested loops. Therefore the following cell.

</span>


In [12]:
from itertools import product

query_summaries = []
for region,stat in product(bl_codes,stats_lst):
    q = Query.region(region)
    q.add_field(stat)
    query_result = q.results()
    query_summaries.append(summarize_result(region,stat,query_result))
        
summary_frame = pd.DataFrame(query_summaries)
summary_frame.head()

/home/konrad/programming/python/correlaid/datenguide-python/datenguidepy/output_transformer.py:130: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  return pd.io.json.json_normalize(statistic_sub_json, sep="_", max_level=1)


,region,statistic,region_name,entries,start_year,end_year,distinct_sources
0,10,PLZ005,Saarland,25,2007.0,2019.0,1.0
1,10,TIE003,NaN,0,NaN,NaN,NaN
2,10,BEVSTD,Saarland,31,1995.0,2018.0,2.0
3,11,PLZ005,Berlin,25,2007.0,2019.0,1.0
4,11,TIE003,NaN,0,NaN,NaN,NaN


<span style="color: blue">
    
## Suggested next steps

The following next steps are suggested to deal with the problem that getting summaries for all statistics
and regions will take very such that one will possibly not want to do it in a single go (possibly
the datenguide API will also block as at one point due to too many requests in too short of a time).
With the suggestions below one could summarize a subset of regions and statistics and pick up where
one left of at a later point in time.
    
- writing summary information dataframe to disk
- reading summary information dataframe from disk
- append to existing summary dataframe information
- condition summary calculation in the loop on summaries not already being present in summary information dataframe
- optionally use the `Query.all_regions` function instead of the `Query.region` function. Although the returned results should not differ the former sends a single query to the API that can query several regions at once. This has the advanatage of reducing the API requests, in case this becomes an issue. It has the disadvantage that regions can only be specified indirectly by means of nuts level/lau and the parent region.
</span>